.apply method execute for every value in series , so avoid it 

In [5]:
import pandas as pd 
url = 'https://github.com/arunadas/effective-pandas/raw/main/data/vehicles.csv.zip'
df = pd.read_csv(url,dtype='unicode')
city_mpg = df.city08.astype(int)

In [6]:
def gt20(val):
    return val > 20 

In [13]:
%%timeit
city_mpg.apply(gt20)

2.51 ms ± 17.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
#broadcast method 
%timeit city_mpg.gt(20)

18 µs ± 109 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
